# Projeto 1 - Ciência dos Dados

#### Nome: Andresa Bicudo

#### Nome: Leonardo Malta

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [78]:
# Importanto bibliotecas

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

In [79]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\user\Documents\Insper\2° semestre\CDados\Projetos\Projeto1_Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [80]:
dados = pd.read_excel('Corona.xlsx')

In [81]:
dados.head(5)

,Relevância,Mensagem
0,1,rt @joaopauloz_: saber que algumas pessoas com...
1,0,rt @thewayjane: se a campeã não sair daqui o c...
2,1,"na espanha tá feio o corona, tenho medo de que..."
3,0,"rt @george_fredson: dos mesmos criadores de ""a..."
4,0,as primeiras que vão pega corona vírus aqui na...


___
## Classificador automático de sentimento


#### Faça aqui uma descrição do seu tema e o que considerou como relevante ou não relevante na classificação dos tweets.

Em meio a essa grande pandemia do COVID-19 (Corona Vírus), é função Ministério da Saúde auxiliar a população com informações de prevenção e de cuidados cajo haja suspeita de contaminação, logo, tweets informativos são relevantes, uma vez que se pode evitar compartilhamento de "fake-news" e manter as recomendações verdadeiras. Ademais, também é essencial que o governo saiba da ocorrência de eventuais eventos que promovem aglomerações e de pessoas que banalizam a situação, a fim de tomar as medidas necessárias para conter o avanço dessa doença. Por fim, como o assunto está sendo extremamente comentado, muitos memes estão sendo feitos, logo, é preciso filtrá-los, recebendo a classificação de não relevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [82]:
# Transformando dados em variaveis qualitativas
dados.loc[:,'Mensagem'] = dados['Mensagem'].astype('category')
dados.loc[:,'Relevância'] = dados['Relevância'].astype('category')
dados.Relevância.cat.categories = ['Irrelevante', 'Relevante']
dados.head(5)

,Relevância,Mensagem
0,Relevante,rt @joaopauloz_: saber que algumas pessoas com...
1,Irrelevante,rt @thewayjane: se a campeã não sair daqui o c...
2,Relevante,"na espanha tá feio o corona, tenho medo de que..."
3,Irrelevante,"rt @george_fredson: dos mesmos criadores de ""a..."
4,Irrelevante,as primeiras que vão pega corona vírus aqui na...


In [83]:
# Separando dados em duas tabelas: Relevantes e Irrelevantes 
tweets_relevantes = dados.loc[dados['Relevância'] == 'Relevante', :]
tweets_irrelevantes = dados.loc[dados['Relevância'] == 'Irrelevante', :]

# Teste de seperação de mensagem
tweets_irrelevantes.Mensagem[1]

'rt @thewayjane: se a campeã não sair daqui o corona já pode cancelar o bbb #bbb20 https://t.co/z4vvf9qn66'

In [84]:
# Criando listas com os índices dos Tweets relevante e irrelevantes
lista_indices_relevantes = []
lista_indices_irrelevantes = []

for i in range(len(dados.Mensagem)):
    if dados.Relevância[i] == 'Relevante':
        lista_indices_relevantes.append(dados.index[i])
    elif dados.Relevância[i] == 'Irrelevante':
        lista_indices_irrelevantes.append(dados.index[i])

In [85]:
# Função de limpeza que troca alguns sinais, menções e links por espaços, e coloca em uma lista.
# Link de consulta para fazer essa função: 
# https://stackoverflow.com/questions/54733828/remove-twitter-mentions-from-pandas-column

def cleanup(tweet):
    tweet_sem_mencoes = re.sub("@[A-Za-z0-9]+","", tweet) # Retirando menções
    tweet_sem_links = re.sub("https[A-Za-z0-9]+","", tweet_sem_mencoes) # Retirando links
    punctuation = '[_ ! - . : ? ; @]' # Sinais que serão substituidos por espaços
    pattern = re.compile(punctuation)
    tweet_final = re.sub(pattern, ' ', tweet_sem_links) # Retirando sinais
    lista_tweet_final = tweet_final.split()
    return lista_tweet_final

# Teste do uso da função de limpeza
my_str = cleanup(tweets_relevantes.Mensagem[0])
print(my_str)

['rt', 'saber', 'que', 'algumas', 'pessoas', 'com', 'o', 'corona', 'vírus', 'estão', 'circulando', 'pelas', 'ruas', 'é', 'só', 'mais', 'uma', 'prova', 'da', 'depravação', 'do', 'homem', 'e', 'o', 'se…']


In [93]:
# Colocando todas a palavras relevantes em uma lista, usando a lista de índices criadas anteriormente:
todas_palavras_relevantes = []
todas_palavras_irrelevantes = []

for i in range(len(lista_indices_relevantes)):
    todas_palavras_relevantes.append(cleanup(tweets_relevantes.Mensagem[lista_indices_relevantes[i]]))

for p in range(len(lista_indices_irrelevantes)):
    todas_palavras_irrelevantes.append(cleanup(tweets_irrelevantes.Mensagem[lista_indices_irrelevantes[p]]))

# Trasnformando uma lista de listas em uma única lista:
palavras_relevantes = []
palavras_irrelevantes = []

def Listas2lista(listadelistas):
    nova_lista = []
    for sublista in listadelistas:
        for item in sublista:
            nova_lista.append(item)
    return nova_lista

palavras_relevantes = Listas2lista(todas_palavras_relevantes)
palavras_irrelevantes = Listas2lista(todas_palavras_irrelevantes)

palvras_relevantes_data = pd.Series(palavras_relevantes)
palvras_irrelevantes_data = pd.Series(palavras_irrelevantes)

In [96]:
# Tabela de frequências absolutas:
tabela_relevantes = palvras_relevantes_data.value_counts()
tabela_irrelevantes = palvras_irrelevantes_data.value_counts()
tabela_irrelevantes

corona                         473
o                              235
rt                             223
de                             189
vírus                          160
que                            154
a                              142
eu                             133
não                            118
e                              117
do                             110
com                            106
https                           95
//t                             94
é                               86
se                              70
virus                           61
da                              61
pra                             54
em                              52
no                              51
vai                             51
por                             45
esse                            44
minha                           42
tá                              42
um                              40
q                               40
só                  

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**